# load tons of datasets (~60,000 RNAseq samples)

In [1]:
from taigapy import TaigaClient
tc = TaigaClient()

from depmapomics import tracker as track
from depmapomics import expressions

import dalmatian as dm
from gsheets import Sheets
import pandas as pd

MY_ID = '~/.client_secret.json'
MYSTORAGE_ID = "~/.storage.json"

Sheets.from_files(MY_ID, MYSTORAGE_ID)
#autoreload
%load_ext autoreload
%autoreload 2

## CCLE + TCGA

In [116]:
# load from taiga public (figshare link)
# load internal expression,
# latest version can be found at https://depmap.org/portal/download/
# can also be loaded like so pd.read_csv('gs://ccle_default_params/celligner_ex/CCLE_expression.csv.gz', index_col=0)
CCLE_expression = tc.get(name='internal-21q3-fe4c',
                         file='CCLE_expression_full')  #40,000x1,500


# load  TCGA expression
# this dataset was generated from  ,using this script: 
# caan be found here: pd.read_csv('gs://ccle_default_params/celligner_ex/TCGA_expression.csv.gz', index_col=0)
TCGA_expression = tc.get(name='celligner-input-9827',
                       file='tumor_expression') # 40,000x13,000

No dataset version provided. Using version 16.
No dataset version provided. Using version 1.


In [23]:
len(CCLE_annotation[(CCLE_annotation.blacklist==0)&(CCLE_annotation.version>1)&(CCLE_annotation.datatype=="rna")]) 
# we have .. replicates in CCLE

72

In [342]:
# loading annotations
CCLE_annotation = track.getTracker() # the function uses pygsheets to load this: REFSHEET_URL=https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY
# Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)
# you can also get it from pd.read_csv('gs://ccle_default_params/celligner_ex/CCLE_annotation.csv.gz', index_col=0)


# can be loaded from 
# pd.read_csv('gs://ccle_default_params/celligner_ex/TCGA_annotation.csv.gz', index_col=0)
TCGA_annotation = tc.get(name='celligner-input-9827',
                         file='tumor_annotations') # generated manually 

No dataset version provided. Using version 1.


In [446]:
CCLE_annotation.iloc[0, :-25]

arxspan_id                                                        ACH-001188
version                                                                    1
sm_id                                                                    NaN
PDO                                                                      NaN
datatype                                                                 wgs
size                                                           75505480632.0
stripped_cell_line_name                                               SHSY5Y
participant_id                                                   PT-VWVFq8aY
cellosaurus_id                                                     CVCL_0019
synonyms                   SH-Sy5y; SHSY5Y; SHSY-5Y; SK-SH-SY5Y; SY5Y; SH...
bam_public_sra_path                                                      NaN
internal_bam_filepath         gs://cclebams/wgs_hg38/CDS-rIh3Gy.wgs_ccle.bam
internal_bai_filepath         gs://cclebams/wgs_hg38/CDS-rIh3Gy.wgs_ccle.bai

In [ ]:
rename = {
    "stripped_cell_line_name": "sample_id", 
    "primary_disease": "disease_type", 
    "subtype": 'disease_subtype', 
    'sampleID': 'sample_id', 
    'age_at_dx':"age", 
    "gender": "sex",
    "site_id": "site_id", 
    "lineage": "primary_site", 
    "disease": "disease_type", 
    "subtype": "disease_subtype",
    'Tumor_type': 'tumor_type', 
    'Sample_type': 'sample_type', 
    'RNA_Seq_cancertype': 'disease_type', 
    "CCLF_ID": 'sample_id',
    "Sequencing on Tissue or Cell model? (MT confirm)": 'tissue_type', 
    "Days to First Agg": 'exp_date', 
    "Contamination % (First Agg)": 'contamination', 
    "collection": 'origin', 
    "Original Material Type": 'history',
    "sampleID": 'sample_id',
    'lineage': 'tissue_type',
    'subtype': "disease_type",
    "type": "cell_type",
    "Sex": "sex",
    'Phase':'stage',
    'sample_source': 'patient_id',
    'tc': 'tumor_purity'

In [343]:
CCLE_annotation = CCLE_annotation.rename(columns=rename)[['origin', 'sequencing_type', 'doublingt','hasebv'] + list(rename.values())]
CCLE_annotation['method']="bulk"
CCLE_annotation['cell_type']="historical_CL"

NameError: name 'rename' is not defined

In [440]:
TCGA_annotation.iloc[0]

sampleID                      TH27_1241_S01
age_at_dx                               NaN
pedaya                              Unknown
gender                              unknown
site_id                                TH27
site_donor_id                          None
site_sampleid                          None
lineage              central_nervous_system
disease                        Brain Cancer
TCGA_disease                           None
TCGA_primary_site                      None
sample_type                            None
CPE                                     NaN
ABSOLUTE                                NaN
Cancer type                            None
subtype                              glioma
UMAP_1                             5.433872
UMAP_2                            10.000675
cluster                                  52
Name: 0, dtype: object

In [40]:
TCGA_annotation = TCGA_annotation.rename(columns=rename)[rename.values()]
TCGA_annotation['method']="bulk"
TCGA_annotation['cell_type']="tumor"
TCGA_annotation['metastasis']="Primary"

## CCLF

In [21]:
cclf_orga_info = tc.get(name='cclf-organoids-c23d', version=1, file='cclf_orga_info')
cclf_orga_info = cclf_orga_info.rename(columns=rename)[rename.values()]
cclf_orga_rnaseq = tc.get(name='cclf-organoids-c23d', version=1, file='cclf_orga_rnaseq').T # 40,000x24
cclf_orga_info.index = [i.split("_")[1] for i in cclf_orga_info.sample_id]
cclf_orga_rnaseq.index = [i.split('_')[0][:-1] for i in cclf_orga_rnaseq.index]

NameError: name 'rename' is not defined

In [536]:
cclf_orga_info

,Genomic_Seq,RNA_Seq_cancertype,RNA_Seq_marker,CCLF_ID,Diagnosis,Subtype_patient,Tumor_type,Sample_type
0,Pending,BREAST_BASAL,None,CCLF_cRCRF1048,Advanced breast cancer,"ER/PR+, HER2-",Metastatic,Pleural effusion
1,Pending,BREAST_BASAL,None,CCLF_cRCRF1060,Invasive breast carcinoma,"ER+/PR-, Her2 IHC 1+",Metastatic,Pleural effusion
2,Pending,BREAST_BASAL,None,CCLF_KL1337,Breast met to brain,Pending,Metastatic,Brain met resection
3,Pending,BREAST_BASAL/LUMINAL,None,CCLF_KL1310,Breast met to brain,Pending,Metastatic,None
4,Pending,BREAST_LUMINAL,ER+,CCLF_KL1271,Breast met to brain,Pending,Metastatic,Brain met resection
5,Pending,BREAST_LUMINAL,ER+,CCLF_KL1333,Invasive breast carcinoma,Pending,Metastatic,Cryopreserved tissue
6,Pending,BREAST_LUMINAL,ER+,CCLF_KL1351,Breast met to brain,Pending,Metastatic,Brain met resection
7,Pending,BREAST_BASAL,None,CCLF_cRCRF1092,Advanced breast cancer,"ER/PR+, HER2 negative",Metastatic,Pleural effusion
8,Pending,BREAST_LUMINAL,HER2+,CCLF_KL1312,Breast met to brain,Pending,Metastatic,Brain met resection
9,Pending,FIBROBLAST,None,CCLF_cRCRF1038,Advanced breast cancer,"ER/PR+, HER2-",Metastatic,Pleural effusion


In [ ]:
cclf_orga_info['sequencer'] =
cclf_orga_info['method'] =

In [480]:
#cclf other
cclfrna = dm.WorkspaceManager("nci-mimoun-bi-org/CCLF_RNA_2_0").get_samples() #40,000x160

cclfrna_anno = cclfrna[["external_id_rna"]].replace({'NA': np.nan})

cclfrna_annot = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/1O9IV_v2vMbebkk_KDWu3LdKBQ16c8lThJKiiWvRxMUo").sheets[2].to_frame()

cclfrna_annot2 = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/1O9IV_v2vMbebkk_KDWu3LdKBQ16c8lThJKiiWvRxMUo").sheets[3].to_frame()
# get it  from https://docs.google.com/spreadsheets/d/1O9IV_v2vMbebkk_KDWu3LdKBQ16c8lThJKiiWvRxMUo and get 

In [392]:
#files, failed, _, _, lowqual, _ = await expressions.postProcess("nci-mimoun-bi-org/CCLF_RNA_2_0", "all_samples", samplesetToLoad = "all_samples", compute_enrichment=False, trancriptLevelCols = ['rsem_transcripts_expected_count', 'rsem_transcripts_tpm'], geneLevelCols = ["rsem_genes_tpm", "rsem_genes_expected_count"], save_output="data/")
#cclfrna = files['rsem_genes_tpm']
cclfrna = pd.read_csv('data/expression_genes_tpm.csv', index_col=0)

In [481]:
ina = (cclfrna_annot2['Passage Number'].isna() | (cclfrna_annot2['Passage Number']=="Unknown")) & ~(cclfrna_annot2["Passage Number on Receipt"].isna() | (cclfrna_annot2["Passage Number on Receipt"]=="Unknown"))
cclfrna_annot2.loc[ina, "Passage Number"] = cclfrna_annot2.loc[ina, "Passage Number on Receipt"].values

ina = (cclfrna_annot2['Gender'].isna() | (cclfrna_annot2['Gender']=="Unknown")) & ~(cclfrna_annot2["Gender.1"].isna() | (cclfrna_annot2["Gender.1"]=="Unknown"))
cclfrna_annot2.loc[ina, "genderA"] = cclfrna_annot2.loc[ina, "Gender.1"].values

ina = (cclfrna_annot2['Gender'].isna() | (cclfrna_annot2['Gender']=="Unknown")) & ~(cclfrna_annot2["FP Gender"].isna() | (cclfrna_annot2["FP Gender"]=="Unknown"))
cclfrna_annot2.loc[ina, "Gender"] = cclfrna_annot2.loc[ina, "FP Gender"].values

ina = (cclfrna_annot2['Race'].isna() | (cclfrna_annot2['Race']=="Unknown")) & ~(cclfrna_annot2["Ethnicity"].isna() | (cclfrna_annot2["Ethnicity"]=="Unknown"))
cclfrna_annot2.loc[ina, "Race"] = cclfrna_annot2.loc[ina, "Ethnicity"].values


In [537]:
cclfrna_annot2.iloc[0]

Age                              NaN
Gender                           NaN
Sample Type                   Normal
Tumor Type                       NaN
Tissue Site                      NaN
Tumor Percent                    NaN
Primary Disease    Mouse Fibroblasts
Race                             NaN
Culture Medium                   NaN
Passage Number                   NaN
index                            NaN
Name: 3T3/J2_CL, dtype: object

In [538]:
cclfrna_annot.iloc[0]

Sequencing on Tissue or Cell model? (MT confirm)                                           Cell model
External ID for BAM                                                                         AB002T_BT
Product                                             Tru-Seq Strand Specific Large Insert RNA Seque...
Original Material Type                                                        Cells:Cell Line, Viable
Collaborator Participant ID                                                                     AB002
Aggregated                                                                                        0.0
Actual Seq Technology                                                                             NaN
Contamination %                                                                                  1.07
Age                                                                                           Unknown
Gender                                                                            

In [482]:
cclfrna_annot2 = cclfrna_annot2.set_index('Collaborator Sample ID')[["Age",
"Gender",
"Tumor Type",
"Tissue Site",
"Primary Disease",
"Race",
"Culture Medium",
"Passage Number",]]

In [483]:
cclfrna_annot = cclfrna_annot[[
 'Sequencing on Tissue or Cell model? (MT confirm)',
 'External ID for BAM',
 'Product',
 'RIN',
 'Collaborator Sample ID',
 'Original Material Type',
 'Collaborator Participant ID',
 'Aggregated',
 'Actual Seq Technology',
 'Contamination %',
]].set_index('Collaborator Sample ID', drop=True)

In [522]:
for val in h.dups(cclfrna_annot2.index):
    for i in range(len(cclfrna_annot2.loc[val])-1):
        if cclfrna_annot2.loc[val].iloc[0].isna().sum() > cclfrna_annot2.loc[val].iloc[i+1].isna().sum():
            cclfrna_annot2.iloc[np.argwhere(cclfrna_annot2.index == val).flatten()[0]] = cclfrna_annot2.loc[val].iloc[i+1].values
cclfrna_annot2 = cclfrna_annot2[~cclfrna_annot2.index.duplicated(keep='first')]

In [525]:
for val in h.dups(cclfrna_annot.index):
    for i in range(len(cclfrna_annot.loc[val])-1):
        if cclfrna_annot.loc[val].iloc[0].isna().sum() > cclfrna_annot.loc[val].iloc[i+1].isna().sum():
            cclfrna_annot.iloc[np.argwhere(cclfrna_annot.index == val).flatten()[0]] = cclfrna_annot.loc[val].iloc[i+1].values
cclfrna_annot = cclfrna_annot[~cclfrna_annot.index.duplicated(keep='first')]

In [526]:
cclfrna_annot = pd.concat([cclfrna_annot, cclfrna_annot2], axis=1)

In [528]:
for i, val in cclfrna_annot.iterrows():
    cclfrna_anno.loc[cclfrna_anno.external_id_rna==i, cclfrna_annot.columns] = val.values
del cclfrna_annot

## MET500 and PDXs

In [544]:
met500_meta.iloc[0]

Sample_id        ES_5001-capt-SI_5013-C0LAMACXX
sample_type                               tumor
sample_source                           ES_5001
dataset                                    mctp
tissue                                   breast
cohort                                     BRCA
run.id           ES_5001-capt-SI_5013-C0LAMACXX
idx                                         461
test                                      False
tc                                         0.93
biopsy_tissue                             brain
Name: 0, dtype: object

In [606]:
tcga_dict = {
"LAML":	"Acute Myeloid Leukemia",
"ACC":	"Adrenocortical carcinoma",
"BLCA":	"Bladder Urothelial Carcinoma",
"BOCA":	"Bone Cancer",
"LGG":	"Brain Lower Grade Glioma",
"BRCA":	"Breast invasive carcinoma",
"CESC":	"Cervical squamous cell carcinoma and endocervical adenocarcinoma",
"CHOL":	"Cholangiocarcinoma",
"CLLE":	"Chronic Lymphocytic Leukemia",
"CMDI":	"Chronic Myeloid Disorders",
"COAD":	"Colon adenocarcinoma",
"COLO":	"Colorectal Cancer",
"COADREAD":	"Colorectal cancer",
"EOPC":	"Early Onset Prostate Cancer",
"ESAD":	"Esophageal Adenocarcinoma",
"ESCA":	"Esophageal carcinoma",
"CHOL":	"Gallbladder cancer",
"GBM":	"Glioblastoma multiforme",
"HNSC":	"Head and Neck squamous cell carcinoma",
"KDNY":	"Kidney Cancer",
"KICH":	"Kidney Chromophobe",
"KIRC":	"Kidney renal clear cell carcinoma",
"KIRP":	"Kidney renal papillary cell carcinoma",
"LIRI":	"Liver Cancer",
"LICA":	"Liver Cancer",
"LINC":	"Liver Cancer",
"HCC":	"Liver hepatocellular carcinoma",
"LIHC":	"Liver hepatocellular carcinoma",
"LGG":	"Lower Grade GLioma",
"LUNG":	"Lung Cancer",
"LUAD":	"Lung adenocarcinoma",
"LUSC":	"Lung squamous cell carcinoma",
"DLBC":	"Lymphoid Neoplasm Diffuse Large B-cell Lymphoma",
"MCTP":	"MCTP",
"MALY":	"Malignant Lymphoma",
"MESO":	"Mesothelioma",
"NBL":	"Neuroblastoma",
"ORCA":	"Oral Cancer",
"MISC":	"Other Cancer",
"OV":	"Ovarian serous cystadenocarcinoma",
"PACA":	"Pancreatic Cancer",
"PAEN":	"Pancreatic Cancer Endocrine neoplasms",
"PAAD":	"Pancreatic adenocarcinoma",
"PBCA":	"Pediatric Brain Cancer",
"PCPG":	"Pheochromocytoma and Paraganglioma",
"PRAD":	"Prostate adenocarcinoma",
"READ":	"Rectum adenocarcinoma",
"RECA":	"Renal Cancer",
"SARC":	"Sarcoma",
"SECR":	"Secretory Cancer",
"SKCM":	"Skin Cutaneous Melanoma",
"STAD":	"Stomach adenocarcinoma",
"TGCT":	"Testicular Germ Cell Tumor",
"TGCT":	"Testicular Germ Cell Tumors",
"THYM":	"Thymoma",
"THYM":	"Thymoma",
"THCA":	"Thyroid carcinoma",
"UCS":	"Uterine Carcinosarcoma",
"UCEC":	"Uterine Corpus Endometrial Carcinoma",
"UVM":	"Uveal Melanoma",
"ACC":	"adrenocortical carcinoma",
}

In [607]:
met500_meta = met500_meta.replace({"cohort": tcga_dict})

In [542]:
# met500 
met500_meta = tc.get(name='met500-fc3c', file='met500_meta')
met500_TPM = tc.get(name='met500-fc3c', file='met500_TPM') #20,979x868 matrix

#Novartis_PDX
Novartis_PDX_ann = tc.get(name='pdx-data-3d29', file='Novartis_PDX_ann')
Novartis_PDX_TPM = tc.get(name='pdx-data-3d29', file='Novartis_PDX_TPM').T # 38,087x445

#pediatric_PDX
pediatric_PDX_ann = tc.get(name='pdx-data-3d29', file='pediatric_PDX_ann')
pediatric_PDX_TPM = tc.get(name='pdx-data-3d29', file='pediatric_PDX_TPM') #80,000x250

No dataset version provided. Using version 1.
No dataset version provided. Using version 1.
No dataset version provided. Using version 1.
No dataset version provided. Using version 2.
No dataset version provided. Using version 2.
No dataset version provided. Using version 2.
No dataset version provided. Using version 2.


In [ ]:
met500_meta = met500_meta.rename(columns={**rename, **{'subtype': "disease_type"}}).set_index('sample_id', drop=True)[rename.values()]

In [ ]:
met500_meta['sequencer'] = ""
met500_meta['method'] = ""

In [347]:
pediatric_PDX_ann.iloc[0]

sampleID                                                                                  ALL-102
lineage                                                                                     blood
subtype                                                                                       ALL
Histology                                                                                     ALL
Histology.Detailed                                                                     Ph-likeALL
Histology-Detailed2                                                                    Ph-likeALL
Molecular-Subtype-Brain                                                                      None
PI                                                                                           Lock
Sex                                                                                          Male
Phase                                                                                   Diagnosis
Age                 

In [70]:
[(i.split('me patient as ')[-1].split(' (')[0],v) if type(i) is str and 'ame patient' in i else '' for v, i in pediatric_PDX_ann[["sampleID","Other_info1"]].values]

[('ALL-105', 'ALL-102'),
 ('ALL-102', 'ALL-105'),
 '',
 '',
 '',
 '',
 ('ALL-102', 'ALL-115'),
 '',
 '',
 '',
 ('ALL-46', 'ALL-121'),
 '',
 ('ALL-58', 'ALL-123'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('ALL-121', 'ALL-46'),
 '',
 '',
 '',
 '',
 '',
 ('ALL-123', 'ALL-58'),
 '',
 ('ALL-25', 'ALL-61'),
 '',
 '',
 ('ALL-81', 'ALL-80'),
 ('ALL-80', 'ALL-81'),
 '',
 ('ALL-32', 'ALL-90'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('ALL-61', 'ALL-25'),
 '',
 '',
 '',
 ('ALL-90', 'ALL-32'),
 '',
 '',
 '',
 '',
 '',
 '',
 ('ALL-83', 'ALL-82'),
 ('ALL-82', 'ALL-83'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('COG-N-453x', 'COG-N-452x'),
 ('COG-N-452x', 'COG-N-453x'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('COG-N-623x', 'COG-N-603x'),
 ('COG-N-618x', 'COG-N-619x'),
 '',
 ('COG-N-603x', 'COG-N-623x'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('98

In [ ]:
pediatric_PDX_ann['participant_id'] = pediatric_PDX_ann.index

#created frrom manual inspection
samepatient = [('NCH-CA-2', 'NCH-CA-1'), ('ALL-105', 'ALL-102', "ALL-115"), ('ALL-46', 'ALL-121'), ('ALL-25', 'ALL-61'), ('ALL-81', 'ALL-80'), ('ALL-32', 'ALL-90'), ('ALL-58', 'ALL-123'), ('ALL-82', 'ALL-83'), ("COG-N-623x", "COG-N-603x"), ("COG-N-453x","COG-N-452x"), ("COG-N-618x", "COG-N-619x"), ('22909PNET', '9850PNET'), ('OS-34', 'OS-34-SJ'), ('OS-36', 'OS-36-SJ', 'OS-32'),  ('Rh-30R', 'Rh-30')]

for val in samepatient:
    for i in val[1:]:
        pediatric_PDX_ann.loc[i, 'participant_id']=val[0]
        
pediatric_PDX_ann['age'] = ['adult' if i =='Adult' else 'child' for i in pediatric_PDX_ann['Other_info1']]

pediatric_PDX_ann.rename(columns={**rename, **{'subtype': "disease_type"}})[rename.values()]

In [ ]:
pediatric_PDX_ann['sequencer'] = ""
pediatric_PDX_ann['method'] = ""

In [350]:
Novartis_PDX_ann.iloc[0]

sampleID           0931HXXTM
lineage             pancreas
subtype     ductal_carcinoma
type                     PDX
Name: 0, dtype: object

In [ ]:
Novartis_PDX_ann = Novartis_PDX_ann.rename(columns=rename).set_index('sample_id', drop=True)

In [ ]:
Novartis_PDX_ann['sequencer'] = ""
Novartis_PDX_ann['method'] = ""

## tumor inf elife

In [11]:
elife_tumorinf = tc.get(name='tumor-infiltration-3307', version=1, file='elife_tumorinf')

In [ ]:
elife_tumorinf = elife_tumorinf.rename(columns={"Bcells": "B-cell", "CAFs": "CAF", "CD4_Tcells": "CD4_T-cells", "CD8_Tcells": "CD8_T-cells","macrophage": "macrophage", "Endothelial": "endothelial", "NKcells": "NK-cell"})

In [13]:
elife_tumorinf_ann = pd.DataFrame()
elife_tumorinf_ann["cell_type"] = "normal"
elife_tumorinf_ann["tissue_type"] = elife_tumorinf.columns
elife_tumorinf_ann["sample_ID"] = elife_tumorinf.columns
melanoma_ann['sequencer'] = ""
melanoma_ann['method'] = ""
melanoma_ann['age'] = ""
melanoma_ann['sex'] = ""

## tirosh's melanoma

In [4]:
melanoma = tc.get(name='tirosh-melanoma-scrnaseq-60f0', file='melanoma')

No dataset version provided. Using version 1.


In [19]:
melanoma.columns = [i.replace('-', '_').replace('Cy', "CY").replace('cy', "CY").replace('CY88C', 'CY88_C').replace('CY89A', "CY89_A").replace('CY89C', 'CY89_C').replace('CY89F', 'CY89_F').replace('CY89N', 'CY89_N').replace('CY94C', 'CY94_C') for i in melanoma.columns]

In [20]:
melanoma_ann = pd.DataFrame()

typ={1:"normal", 2:"tumor",0: np.nan}
orig={1:"melanoma", 2:"B-cell", 3: "macrophage", 4: "endothelial", 5: "CAF", 6:"NK-cell", 0: np.nan}

melanoma_ann['age'] = [int(i) for i in melanoma.loc['tumor']]
melanoma_ann["cell_type"] = [typ[int(i)] for i in melanoma.loc['malignant(1=no,2=yes,0=unresolved)']]
melanoma_ann['tissue_type'] = [orig[int(i)] for i in melanoma.loc['non-malignant cell type (1=T,2=B,3=Macro.4=Endo.,5=CAF;6=NK)']]
melanoma_ann['name'] = [i.split('_')[0] for i in melanoma.columns]
melanoma_ann['sample_id'] = melanoma.columns
melanoma_ann['other'] = [i.split('_')[-2] for i in melanoma.columns]
melanoma_ann['sequencer'] = 
melanoma_ann['method'] = 

## GTEX

In [613]:
from anndata import AnnData, read_h5ad

In [9]:
#! curl https://storage.googleapis.com/gtex_analysis_v9/snrna_seq_data/GTEx_8_tissues_snRNAseq_atlas_071421.public_obs.h5ad --output temp/gtex_8_atlas_public.h5ad

## GTEX additional
https://storage.googleapis.com/gtex_external_datasets/eyegex_data/rna_seq_data/EyeGEx_retina_combined_genelevel_expectedcounts_byrid_nooutlier.tpm.matrix.gct
    
https://storage.googleapis.com/gtex_external_datasets/eyegex_data/annotations/EyeGEx_meta_combined_inferior_retina_summary_deidentified_geo_ids.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1810M  100 1810M    0     0   126M      0  0:00:14  0:00:14 --:--:--  126M


In [614]:
gtex_v9 = read_h5ad("temp/gtex_8_atlas_public.h5ad") #209,126 × 17,695

In [623]:
gtex_v9.obs = gtex_v9.obs[["Age_bin","Sex","Sample ID", "Participant ID", "RIN score from PAXgene tissue Aliquot", "Tissue"]].rename(columns={**rename, **{"tissue": "tissue_type", "Tissue": "collection_site"}})

n_genes                                                                    2658
fpr                                                                         0.1
tissue                                                           skeletalmuscle
prep                                                                        CST
individual                                                                   01
nGenes                                                                     2902
nUMIs                                                                   11544.0
PercentMito                                                             0.07623
PercentRibo                                                            0.051195
Age_bin                                                                   51-60
Sex                                                                        Male
Sample ID                                              GTEX-1HSMQ-5011-SM-GKSJH
Participant ID                          

In [ ]:
gtex_v9.obs['sequencer']="Illumina TrueSeq"

In [ ]:
gtex_add = # 80,000 x 500

## THEIS LAB scRNAseq datasets

In [ ]:
https://theislab.github.io/sfaira-portal/Datasets #50,000x13,000


## HCMI

In [620]:
# HCMI dataset
# Code to generate this dataset can be found here:
# https://github.com/broadinstitute/hcmi-processing/blob/main/hcmi-rna-analysis-210226.ipynb
hcmi_ltpm = tc.get(name='hcmi-data-ac4b', file='hcmi_ltpm').T # 60486 x 157
hcmi_sample_info = tc.get(name='hcmi-data-ac4b', file='hcmi_sample_info')
#sample_info = tc.get(name='hcmi-data-ac4b', file='sample-info')

No dataset version provided. Using version 7.
No dataset version provided. Using version 7.


In [621]:
set(hcmi_sample_info.Race)

{'Asian', 'Black or African American', None, 'Unknown', 'White'}

In [6]:
hcmi_sample_info.iloc[0]

Case ID                                                                  HCM-CSHL-0092-C25
Primary Site                                                                      Pancreas
Clinical Tumor Diagnosis                                                 Pancreatic cancer
subtype                                                         Adenocarcinoma ductal type
Tissue Status                                                                      Primary
Acquisition Site                                                           Pancreatic head
Gender                                                                                Male
Race                                                                               Unknown
Age At Acquisition (Years)                                                            69.0
Age At Diagnosis (Years)                                                              69.0
Disease Status                                                         Progressive disease

In [5]:
hcmi_sample_info.columns
rename {'Case ID': 'patient_ID', 'Primary Site': 'primary_site', 'subtype', 'lineage', 

Index(['Case ID', 'Primary Site', 'Clinical Tumor Diagnosis', 'subtype',
       'Tissue Status', 'Acquisition Site', 'Gender', 'Race',
       'Age At Acquisition (Years)', 'Age At Diagnosis (Years)',
       'Disease Status', 'Vital Status', 'TNM Stage',
       'Clinical Stage Grouping', 'Histological Grade', 'Has Multiple Models',
       'Neoadjuvant Therapy', 'Chemotherapeutic Drug List Available',
       'Therapy', 'Available Molecular Characterizations',
       'Link To Distributor', 'Type', 'Licensing Required For Commercial Use',
       'Expansion Status', '# Mutated Genes', '# Research Somatic Variants',
       '# Clinical Variants', '# Histo-pathological Biomarkers', 'sampleID',
       'type', 'lineage'],
      dtype='object')

## GDSC

In [ ]:
for i in range(1):
    val = "E-MTAB-3610.raw."+i+".zip"
    ! curl https://www.ebi.ac.uk/arrayexpress/files/E-MTAB-3610/$val -o temp/$val
    ! gunzip temp/$val # 40,000x1200

## L1000 dataset

In [31]:
# you will need python2 
import cmapPy.pandasGEXpress.parse_gct as pg

In [32]:
pg.parse('temp/'+"level5_beta_trt_misc_n8283x12328.gctx")

The given path to the gct file cannot be found. gct_path: temp/level5_beta_trt_misc_n8283x12328.gctx


Exception: The given path to the gct file cannot be found. gct_path: temp/level5_beta_trt_misc_n8283x12328.gctx

In [25]:
# you will need R > 4.0 
# https://www.charlesbordet.com/en/how-to-upgrade-to-R-4-0-0-on-debian/#the-naive-solution
! R -e "if(!requireNamespace('BiocManager', quietly = TRUE)){install.packages('BiocManager', repos='http://cran.us.r-project.org')};BiocManager::install('cmapR');"


R version 4.1.2 (2021-11-01) -- "Bird Hippie"
Copyright (C) 2021 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> if(!requireNamespace("BiocManager", quietly = TRUE)){install.packages("BiocManager", repos="http://cran.us.r-project.org")};BiocManager::install(c("limma"));
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Warning in install.packages("BiocManager", repos = "http://cran.us.r-pr

In [35]:
folder = "gs://ccle_default_params/celligner_ex/"
for val in ["level5_beta_ctl_n58022x12328.gctx",
            "level5_beta_trt_cp_n720216x12328.gctx",
            "level5_beta_trt_misc_n8283x12328.gctx",]:
            #"level5_beta_trt_oe_n34171x12328.gctx",
            #"level5_beta_trt_sh_n238351x12328.gctx",
            #"level5_beta_trt_xpr_n142901x12328.gctx",]:
    cmd = "gsutil cp " + folder + val + " temp/"
    ! $cmd
    res = pg('temp/'+val)

Copying gs://ccle_default_params/celligner_ex/level5_beta_trt_cp_n720216x12328.gctx...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

| [1 files][ 33.1 GiB/ 33.1 GiB]   51.2 MiB/s                                   
Operation completed over 1 objects/33.1 GiB.                                     


TypeError: 'module' object is not callable

## encode

In [644]:
todl = h.fileToList('data/encode_rna.txt')

In [ ]:
todl

In [ ]:
report.iloc[0]

In [643]:
# 40,000 x 1100
report = pd.read_csv('data/encode_report.tsv', sep="\t", skiprows=1)

,ID,Accession,Assay name,Assay title,Target,Target of assay,Target gene symbol,Biosample summary,Biosample term name,Dbxrefs,...,Biosample treatment duration,Biosample treatment duration units,Synchronization,Post-synchronization time,Post-synchronization time units,Replicates,Mixed biosamples,Cellular component,Library construction platform,Library construction method
0,/experiments/ENCSR764OPZ/,ENCSR764OPZ,polyA plus RNA-seq,polyA plus RNA-seq,NaN,NaN,NaN,strain B6NCrl midbrain tissue embryo (10.5 days),midbrain,GEO:GSE88517,...,NaN,NaN,NaN,NaN,NaN,/replicates/40f6ffb3-344d-4654-ae5e-56a57862b6...,NaN,NaN,NaN,NaN
1,/experiments/ENCSR992IIX/,ENCSR992IIX,RNA-seq,total RNA-seq,NaN,NaN,NaN,strain C57BL/6J macrophage chromatin fraction,macrophage,GEO:GSE178056,...,NaN,NaN,NaN,NaN,NaN,/replicates/d392b112-3a81-450e-a263-c6dfebf2a9...,NaN,chromatin,NaN,NaN
2,/experiments/ENCSR460RCF/,ENCSR460RCF,RNA-seq,total RNA-seq,NaN,NaN,NaN,strain C57BL/6J macrophage cytosolic fraction,macrophage,GEO:GSE177319,...,NaN,NaN,NaN,NaN,NaN,/replicates/a1b23c81-ae5c-431d-865e-8be9d15197...,NaN,cytosol,NaN,NaN
3,/experiments/ENCSR049UJU/,ENCSR049UJU,polyA plus RNA-seq,polyA plus RNA-seq,NaN,NaN,NaN,strain B6NCrl heart tissue embryo (10.5 days),heart,GEO:GSE87991,...,NaN,NaN,NaN,NaN,NaN,/replicates/8f26e06a-b808-4696-bc4f-d20821fffb...,NaN,NaN,NaN,NaN
4,/experiments/ENCSR809VYL/,ENCSR809VYL,polyA plus RNA-seq,polyA plus RNA-seq,NaN,NaN,NaN,strain B6NCrl embryonic facial prominence tiss...,embryonic facial prominence,GEO:GSE88554,...,NaN,NaN,NaN,NaN,NaN,/replicates/dad68aad-227f-4446-bdf2-4c5ea8c7ed...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,/experiments/ENCSR763CQW/,ENCSR763CQW,polyA plus RNA-seq,polyA plus RNA-seq,NaN,NaN,NaN,T-cell male adult (37 years),T-cell,GEO:GSM1220574,...,NaN,NaN,NaN,NaN,NaN,/replicates/c1d83dc2-2f73-4281-8962-5875010a3cc1/,NaN,NaN,NaN,NaN
1456,/experiments/ENCSR549DVY/,ENCSR549DVY,polyA plus RNA-seq,polyA plus RNA-seq,NaN,NaN,NaN,stomach tissue female embryo (108 days),stomach,GEO:GSM1220587,...,NaN,NaN,NaN,NaN,NaN,/replicates/9b66601f-b1d4-4710-8fae-b5545f4d09e8/,NaN,NaN,NaN,NaN
1457,/experiments/ENCSR448DCX/,ENCSR448DCX,polyA plus RNA-seq,polyA plus RNA-seq,NaN,NaN,NaN,urinary bladder tissue male child (3 years),urinary bladder,GEO:GSM1067793,...,NaN,NaN,NaN,NaN,NaN,/replicates/f7170bda-475e-42e0-915b-3a3f10751b5c/,NaN,NaN,NaN,NaN
1458,/experiments/ENCSR000AEP/,ENCSR000AEP,polyA plus RNA-seq,polyA plus RNA-seq,NaN,NaN,NaN,K562,K562,GEO:GSE78560,...,NaN,NaN,NaN,NaN,NaN,/replicates/6d52759f-399e-43ed-97c5-073f1858f4...,NaN,NaN,NaN,NaN


## NCI 60

## tumor cell atlas

## other random datasets from SRA

## ICGC

In [642]:
 # 40,000 x 2,000 ALL NONE TCGA ICGC
for i in ["https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/BOCA-FR/exp_seq.BOCA-FR.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/BPLL-FR/exp_seq.BPLL-FR.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/BRCA-KR/exp_seq.BRCA-KR.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/LICA-FR/exp_seq.LICA-FR.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/LIRI-JP/exp_seq.LIRI-JP.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/ORCA-IN/exp_seq.ORCA-IN.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/OV-AU/exp_seq.OV-AU.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/PACA-AU/exp_seq.PACA-AU.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/PACA-CA/exp_seq.PACA-CA.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/PRAD-CA/exp_seq.PRAD-CA.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/PRAD-FR/exp_seq.PRAD-FR.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/CLLE-ES/exp_seq.CLLE-ES.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/MALY-DE/exp_seq.MALY-DE.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/PAEN-AU/exp_seq.PAEN-AU.tsv.gz",
"https://dcc.icgc.org/api/v1/download\?fn\=/current/Projects/RECA-EU/exp_seq.RECA-EU.tsv.gz"]:
    ! wget $i -o data/

--2021-11-05 22:51:05--  https://dcc.icgc.org/api/v1/download?fn=/current/Projects/BOCA-FR/exp_seq.BOCA-FR.tsv.gz
Resolving dcc.icgc.org (dcc.icgc.org)... 206.108.121.75
Connecting to dcc.icgc.org (dcc.icgc.org)|206.108.121.75|:443... connected.
HTTP request sent, awaiting response... 307 
Location: https://download.icgc.org/downloads/static?token=eyJjdHkiOiJKV1QiLCJlbmMiOiJBMTI4R0NNIiwiYWxnIjoiZGlyIn0..7XUhJINwjVhhXTZz.LpJ93RPkZ51FNU6QukNUHbm-nVa1qSc2WHR7U8sZY0rUd6G69hNUsYy5B1IWA7Xy16VIW6txNrALvSkvA4P4ZkZttWIgEJzSMfwU-K2jyEFWwriQYEzB_SEME37oNCeaVg0X0DTvXboKtEtr_wjZBPOmcBfI6kGUwXlPHt1WNFC1u30efOy2sUkZhAyEmzOHhdg-YMp0e0ffXJsSQesSFBbTob1kn1Vbnesgvrh_IVfHmRyYyh8s83mx-IsdMRXknDld1kz30n8nls3DAhu-KkUmhQEUSEWDJXoo5y5v99qHri1jmPdLw3ImHJx9hIcSiW_Fkw1PE2TSHQMJSg_Y.MFd4O7smnObu0_KZBPsoYA [following]
--2021-11-05 22:51:05--  https://download.icgc.org/downloads/static?token=eyJjdHkiOiJKV1QiLCJlbmMiOiJBMTI4R0NNIiwiYWxnIjoiZGlyIn0..7XUhJINwjVhhXTZz.LpJ93RPkZ51FNU6QukNUHbm-nVa1qSc2WHR7U8sZY0rUd6G69hNU

Resolving download.icgc.org (download.icgc.org)... 206.108.121.80
Connecting to download.icgc.org (download.icgc.org)|206.108.121.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘download?fn=%2Fcurrent%2FProjects%2FLIRI-JP%2Fexp_seq.LIRI-JP.tsv.gz’

download?fn=%2Fcurr     [        <=>         ] 169.54M  17.7MB/s    in 13s     

2021-11-05 22:51:36 (13.2 MB/s) - ‘download?fn=%2Fcurrent%2FProjects%2FLIRI-JP%2Fexp_seq.LIRI-JP.tsv.gz’ saved [177770706]

--2021-11-05 22:51:37--  https://dcc.icgc.org/api/v1/download?fn=/current/Projects/ORCA-IN/exp_seq.ORCA-IN.tsv.gz
Resolving dcc.icgc.org (dcc.icgc.org)... 206.108.121.75
Connecting to dcc.icgc.org (dcc.icgc.org)|206.108.121.75|:443... connected.
HTTP request sent, awaiting response... 307 
Location: https://download.icgc.org/downloads/static?token=eyJjdHkiOiJKV1QiLCJlbmMiOiJBMTI4R0NNIiwiYWxnIjoiZGlyIn0..8q_CMr7aFi67DzuS.16oRDouATeR_uHVwY0IhL0swekYYNVqhliPS8baR45Do94Xk

Resolving download.icgc.org (download.icgc.org)... 206.108.121.80
Connecting to download.icgc.org (download.icgc.org)|206.108.121.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘download?fn=%2Fcurrent%2FProjects%2FPRAD-CA%2Fexp_seq.PRAD-CA.tsv.gz’

download?fn=%2Fcurr     [     <=>            ] 113.42M  8.22MB/s    in 15s     

2021-11-05 22:52:21 (7.50 MB/s) - ‘download?fn=%2Fcurrent%2FProjects%2FPRAD-CA%2Fexp_seq.PRAD-CA.tsv.gz’ saved [118928608]

--2021-11-05 22:52:22--  https://dcc.icgc.org/api/v1/download?fn=/current/Projects/PRAD-FR/exp_seq.PRAD-FR.tsv.gz
Resolving dcc.icgc.org (dcc.icgc.org)... 206.108.121.75
Connecting to dcc.icgc.org (dcc.icgc.org)|206.108.121.75|:443... connected.
HTTP request sent, awaiting response... 307 
Location: https://download.icgc.org/downloads/static?token=eyJjdHkiOiJKV1QiLCJlbmMiOiJBMTI4R0NNIiwiYWxnIjoiZGlyIn0..zIV5xwu9L3mXOAcx.6DVoBTUo87d9k8P_BdY6ufQVtIESEpzW1zGF4zwEje76_FZp

Connecting to download.icgc.org (download.icgc.org)|206.108.121.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘download?fn=%2Fcurrent%2FProjects%2FRECA-EU%2Fexp_seq.RECA-EU.tsv.gz’

download?fn=%2Fcurr     [            <=>     ]  81.49M  18.7MB/s    in 4.4s    

2021-11-05 22:52:56 (18.7 MB/s) - ‘download?fn=%2Fcurrent%2FProjects%2FRECA-EU%2Fexp_seq.RECA-EU.tsv.gz’ saved [85451655]



In [4]:
val = pd.read_csv('data/download?fn=%2Fcurrent%2FProjects%2FBOCA-FR%2Fexp_seq.BOCA-FR.tsv.gz', sep='\t')

In [5]:
val

,icgc_donor_id,project_code,icgc_specimen_id,icgc_sample_id,submitted_sample_id,analysis_id,gene_model,gene_id,normalized_read_count,raw_read_count,...,platform,total_read_count,experimental_protocol,alignment_algorithm,normalization_algorithm,other_analysis_algorithm,sequencing_strategy,raw_data_repository,raw_data_accession,reference_sample_type
0,DO50636,BOCA-FR,SP195654,SA607599,A61T5,A61T5_RNAseq,Ensembl,ENSG00000127720,4.94182,358,...,Illumina HiSeq,121982843,Paired End https://www.illumina.com/science/te...,STAR 2.6.0a https://github.com/alexdobin/STAR ...,Cufflinks 2.2.1 http://cole-trapnell-lab.githu...,-,RNA-Seq,EGA,EGAF00002259326:EGAF00002259317:EGAF0000225933...,NaN
1,DO50636,BOCA-FR,SP195654,SA607599,A61T5,A61T5_RNAseq,Ensembl,ENSG00000224440,0.00000,0,...,Illumina HiSeq,121982843,Paired End https://www.illumina.com/science/te...,STAR 2.6.0a https://github.com/alexdobin/STAR ...,Cufflinks 2.2.1 http://cole-trapnell-lab.githu...,-,RNA-Seq,EGA,EGAF00002259326:EGAF00002259317:EGAF0000225933...,NaN
2,DO50636,BOCA-FR,SP195654,SA607599,A61T5,A61T5_RNAseq,Ensembl,ENSG00000261140,0.00000,0,...,Illumina HiSeq,121982843,Paired End https://www.illumina.com/science/te...,STAR 2.6.0a https://github.com/alexdobin/STAR ...,Cufflinks 2.2.1 http://cole-trapnell-lab.githu...,-,RNA-Seq,EGA,EGAF00002259326:EGAF00002259317:EGAF0000225933...,NaN
3,DO50636,BOCA-FR,SP195654,SA607599,A61T5,A61T5_RNAseq,Ensembl,ENSG00000221200,0.00000,0,...,Illumina HiSeq,121982843,Paired End https://www.illumina.com/science/te...,STAR 2.6.0a https://github.com/alexdobin/STAR ...,Cufflinks 2.2.1 http://cole-trapnell-lab.githu...,-,RNA-Seq,EGA,EGAF00002259326:EGAF00002259317:EGAF0000225933...,NaN
4,DO50636,BOCA-FR,SP195654,SA607599,A61T5,A61T5_RNAseq,Ensembl,ENSG00000266468,0.00000,0,...,Illumina HiSeq,121982843,Paired End https://www.illumina.com/science/te...,STAR 2.6.0a https://github.com/alexdobin/STAR ...,Cufflinks 2.2.1 http://cole-trapnell-lab.githu...,-,RNA-Seq,EGA,EGAF00002259326:EGAF00002259317:EGAF0000225933...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3291802,DO234432,BOCA-FR,SP203419,SA607647,B95T5,B95T5_RNAseq,Ensembl,ENSG00000131725,1.93459,810,...,Illumina HiSeq,84394691,Paired End https://www.illumina.com/science/te...,STAR 2.6.0a https://github.com/alexdobin/STAR ...,Cufflinks 2.2.1 http://cole-trapnell-lab.githu...,-,RNA-Seq,EGA,EGAF00002259813:EGAF00002259814,NaN
3291803,DO234432,BOCA-FR,SP203419,SA607647,B95T5,B95T5_RNAseq,Ensembl,ENSG00000219435,12.91860,6,...,Illumina HiSeq,84394691,Paired End https://www.illumina.com/science/te...,STAR 2.6.0a https://github.com/alexdobin/STAR ...,Cufflinks 2.2.1 http://cole-trapnell-lab.githu...,-,RNA-Seq,EGA,EGAF00002259813:EGAF00002259814,NaN
3291804,DO234432,BOCA-FR,SP203419,SA607647,B95T5,B95T5_RNAseq,Ensembl,ENSG00000119509,2.52918,894,...,Illumina HiSeq,84394691,Paired End https://www.illumina.com/science/te...,STAR 2.6.0a https://github.com/alexdobin/STAR ...,Cufflinks 2.2.1 http://cole-trapnell-lab.githu...,-,RNA-Seq,EGA,EGAF00002259813:EGAF00002259814,NaN
3291805,DO234432,BOCA-FR,SP203419,SA607647,B95T5,B95T5_RNAseq,Ensembl,ENSG00000248195,0.00000,0,...,Illumina HiSeq,84394691,Paired End https://www.illumina.com/science/te...,STAR 2.6.0a https://github.com/alexdobin/STAR ...,Cufflinks 2.2.1 http://cole-trapnell-lab.githu...,-,RNA-Seq,EGA,EGAF00002259813:EGAF00002259814,NaN


## st jude

In [ ]:
pd.read_csv('') # 40,000 x 3500


## DUOS datasets

In [ ]:
#https://duos.broadinstitute.org/dataset_catalog

## hartwig

## Pancreas from Sri

## Neurosphere from Keith

# QC

## get the same set of overlapping genes

## compute correlation

- find unknown duplicates
- find problematic duplicates

## analyse  annotation files

- find mismatch annotations
- add missing annotations, need: 
    - sequencer, 
    - expression_type, 
    - media, 
    - origin, 
    - tissue type,
    - disease,
    - sub_disease,
    - cell type, 
    - group, 
    - sex, 
    - age, 
    - contamination
    - organism
- find good set of names for annotations